In [ ]:
!pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='your_api_key_here',
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
!pip install langchain_community

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/retail-associate-ft-nike-princeton/job/R-68473")
page_data = loader.load().pop().page_content
print(page_data)





















Retail Associate, FT - Nike Princeton










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previo

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)


```json
{
  "role": "Retail Associate, FT - Nike Princeton",
  "experience": "Full Time",
  "skills": [
    "Customer service",
    "Communication",
    "Teamwork",
    "Adaptability",
    "Basic math skills",
    "Ability to work in a fast-paced environment"
  ],
  "description": "We believe that if you have a body, you are an athlete, which is why when you join our team as a Retail Associate, you are referred to as an Athlete. Are you ready to embrace it? Let’s do it. Lace Up as a Nike Retail Associate. As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way."
}
```


In [7]:
print(type(res.content))

<class 'str'>


In [9]:
# convert it into json object
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

print(json_res)

{'role': 'Retail Associate, FT - Nike Princeton', 'experience': 'Full Time', 'skills': ['Customer service', 'Communication', 'Teamwork', 'Adaptability', 'Basic math skills', 'Ability to work in a fast-paced environment'], 'description': 'We believe that if you have a body, you are an athlete, which is why when you join our team as a Retail Associate, you are referred to as an Athlete. Are you ready to embrace it? Let’s do it. Lace Up as a Nike Retail Associate. As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}


In [10]:
type(json_res)

dict

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
!pip install chromadb

In [13]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 62.4MiB/s]


In [18]:
links = collection.query(query_texts = ["Experience in python" , "Expertise in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [19]:
job = json_res
job["skills"]

['Customer service',
 'Communication',
 'Teamwork',
 'Adaptability',
 'Basic math skills',
 'Ability to work in a fast-paced environment']

In [20]:
links = collection.query(query_texts = job["skills"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [21]:
job

{'role': 'Retail Associate, FT - Nike Princeton',
 'experience': 'Full Time',
 'skills': ['Customer service',
  'Communication',
  'Teamwork',
  'Adaptability',
  'Basic math skills',
  'Ability to work in a fast-paced environment'],
 'description': 'We believe that if you have a body, you are an athlete, which is why when you join our team as a Retail Associate, you are referred to as an Athlete. Are you ready to embrace it? Let’s do it. Lace Up as a Nike Retail Associate. As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Sahil, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Sahil, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Customer Experience with AtliQ's Automated Solutions

Dear Hiring Manager at Nike,

I came across the job posting for a Retail Associate at Nike Princeton and was impressed by the emphasis on providing exceptional customer service and creating a seamless shopping experience. As a Business Development Executive at AtliQ, I believe our company can help Nike achieve these goals by streamlining business processes and leveraging automated tools.

At AtliQ, we specialize in facilitating the integration of business processes through AI and software consulting. Our expertise in areas like Magento, machine learning, and DevOps can help Nike optimize its retail operations, reduce costs, and enhance overall efficiency. For instance, our Magento portfolio (https://example.com/magento-portfolio) showcases our ability to develop customized e-commerce solutions that can improve customer engagement and drive sales.

Our machine learning capabilities (https://example.com/ml-python-port